## **Configuración de entorno**

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/monografia_UDEA/Codigo/
%run 00_configuracion_entorno.ipynb

Mounted at /content/drive
/content/drive/MyDrive/monografia/Codigo
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: '/content/drive/MyDrive/Test'
/content/drive/MyDrive/monografia/Codigo
Archive:  DataSet.zip


fatal: destination path 'monografia_UDEA' already exists and is not an empty directory.
replace coin_Tether.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


     |████████████████████████████████| 1.4 MB 4.2 MB/s 
     |████████████████████████████████| 9.8 MB 34.8 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


**Selección de la característica de Open e indexación**

In [2]:
bitcoinOpen_Date = bitcoin[["Date","Open"]]
bitcoinOpen_Date.head()

,Date,Open
0,2013-04-29 23:59:59,134.444000
1,2013-04-30 23:59:59,144.000000
2,2013-05-01 23:59:59,139.000000
3,2013-05-02 23:59:59,116.379997
4,2013-05-03 23:59:59,106.250000


Se le da formato a la fecha, especificando que es d/m/a 

In [3]:
bitcoinOpen = bitcoinOpen_Date.copy()
bitcoinOpen['Date'] = pd.to_datetime(bitcoinOpen['Date']).values
bitcoinOpen['Date'] = bitcoinOpen['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
bitcoinOpen.Date = pd.to_datetime(bitcoinOpen.Date, dayfirst = True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

se indexa la fecha

In [4]:
bitcoinOpen.set_index("Date", inplace=True)

Se hace una análisis estadístico sobre la serie de tiempo

In [5]:
bitcoinOpen.describe()

,Open
count,2991.000000
mean,6700.146240
std,11288.043736
min,68.504997
25%,430.445496
50%,2269.889893
75%,8569.656494
max,63523.754869


visualización del Dataframe

In [6]:
bitcoinOpen.head()

,Open
Date,
2013-04-29,134.444000
2013-04-30,144.000000
2013-05-01,139.000000
2013-05-02,116.379997
2013-05-03,106.250000


In [7]:
bitcoinOpen_log10 = bitcoinOpen.apply(lambda x: np.log10(x))

adicionalmente se hace un diferencial para comparar los datos actuales con un año atrás.

In [8]:
bitcoinLog10_diff = bitcoinOpen_log10.Open.diff(365).dropna()

In [9]:
# usemos scikitlearn para normalizar los datos
scaler = MinMaxScaler(feature_range=(0, 1))
bitcoinOpen_log10_Norm = scaler.fit_transform(bitcoinLog10_diff.values.reshape(-1,1))

# Guardar Salida

In [13]:
%cd /content/drive/MyDrive/monografia_UDEA/Salidas

/content/drive/MyDrive/monografia/Codigo/Salidas


In [16]:
df = pd.DataFrame(bitcoinOpen_log10_Norm,columns=['Log10_Norm'])

In [18]:
df.to_csv('dataframe_bitcoin_preprocesado.csv', sep=';', index=False)